In [245]:
print("Test")

Test


In [246]:
import numpy as np
import sympy as sp

In [247]:
nazwa_pliku = 'input.txt'

# Funkcja, która zastępuje przecinki jako separator dziesiętny na kropki - w celu poprawnego wczytania danych za
# pomocą np.loadtxt
def replace_commas_with_periods(filename):
    with open(filename, 'r') as file:
        contents = file.read()
        new_contents = contents.replace(',', '.')
    with open(filename, 'w') as file:
        file.write(new_contents)

replace_commas_with_periods(nazwa_pliku)
coords = np.loadtxt('./' + nazwa_pliku, delimiter=' ')
print(coords)

[[-2.5  16.5 ]
 [-2.   19.  ]
 [-1.5  17.65]
 [-1.   13.8 ]
 [-0.5   8.8 ]
 [ 0.    4.  ]
 [ 0.5   0.75]]


In [248]:
xi = coords[:, 0]
fx = coords[:, 1]



In [249]:
def get_next_column(fx: np.ndarray):
   next_column = np.array([])
   last_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != last_f:
        next_column = np.append(next_column, round((fx[index[0]+1] - f), 2))
   return next_column
print(get_next_column(fx))

[ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]


In [250]:
columns = []

def calculate_cols(column: np.ndarray):
    global columns
    col = get_next_column(column)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols(col)
    else:
        return

calculate_cols(fx)
print(columns)

[array([ 2.5 , -1.35, -3.85, -5.  , -4.8 , -3.25]), array([-3.85, -2.5 , -1.15,  0.2 ,  1.55]), array([1.35, 1.35, 1.35, 1.35])]


In [251]:
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]
Column 2: [-3.85 -2.5  -1.15  0.2   1.55]
Column 3: [1.35 1.35 1.35 1.35]


In [253]:
for column in columns:
    print(column[3])

-5.0
0.2
1.35
